## 누락된 데이터 다루기

### 테이블 형태의 데이터에서 누락된 값 식별

In [21]:
import pandas as pd
from io import StringIO

In [22]:
csv_data = \
"""
A,B,C,D
1.0, 2.0, 3.0, 4.0
5.0, 6.0,,8.0
10.0,11.0,12.0,
"""

In [23]:
df = pd.read_csv(StringIO(csv_data))

In [24]:
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [25]:
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

In [26]:
df.dropna(axis=0)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [27]:
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [28]:
df.dropna(how='all') # 모두 NaN일때 삭제

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [29]:
df.dropna(thresh=4) # 실수가 4보다 작을 때

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [30]:
df.dropna(subset=['C']) # 특정열에 NaN이 있는 경우에 삭제 (C열)

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


### 누락된 값 대체하기

In [31]:
import numpy as np
from sklearn.impute import SimpleImputer

In [32]:
simr = SimpleImputer(missing_values=np.nan, strategy='mean')
simr = simr.fit(df.values)
imputed_data = simr.transform(df.values)
imputed_data


array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [33]:
from sklearn.preprocessing import FunctionTransformer

In [34]:
ftr_simr = FunctionTransformer(lambda X: simr.fit_transform(X.T).T, validate=False)
imputed_data = ftr_simr.fit_transform(df.values)
imputed_data

array([[ 1.        ,  2.        ,  3.        ,  4.        ],
       [ 5.        ,  6.        ,  6.33333333,  8.        ],
       [10.        , 11.        , 12.        , 11.        ]])

## 범주형 데이터 다루기

In [38]:
df = pd.DataFrame([['green', 'M', 10.1,'class1'], ['red','L',13.5,'class2'], ['blue', 'XL', 15.3, 'class1']])
df.columns = ['color','size','price','classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


- 범주형의 문자열 값을 정수형으로 바꾸어 주여야 한다.
- XL = L + 1 = M + 2 라고 가정하자

In [39]:
size_mapping = {'XL' : 3, 'L' : 2, 'M' : 1}
df["size"] = df["size"].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [41]:
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

## 클래스 레이블 인코딩
- 클래스 레이블이 정수로 인코딩 되었을 것이라 라이브러리는 기대함. 사이킷런의 분류 추정기의 대부분은 자체적 클래스 레이블을 정수로 변환하지만, 사소한 실수를 방지하기 위해 클래스 레이블은 배열로 전달하는 것이 좋은 습관이다.

In [42]:
class_mapping = {label:idx for idx, label in enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

In [43]:
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


In [44]:
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [45]:
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

array([0, 1, 0])

In [46]:
class_le.inverse_transform(y)

array(['class1', 'class2', 'class1'], dtype=object)

## One-hot encoding 적용

In [47]:
X = df[['color' , 'size', 'size']].values
color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:, 0])
X

array([[1, 1, 1],
       [2, 2, 2],
       [0, 3, 3]], dtype=object)

In [54]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder

In [56]:
oh_enc = OneHotEncoder(categories='auto')
col_trans = ColumnTransformer([('oh_enc', oh_enc, [0])], remainder='passthrough')
col_trans.fit_transform(X)

array([[0.0, 1.0, 0.0, 1, 1],
       [0.0, 0.0, 1.0, 2, 2],
       [1.0, 0.0, 0.0, 3, 3]], dtype=object)

In [57]:
pd.get_dummies(df[['price','color','size']])

,price,size,color_blue,color_green,color_red
0,10.1,1,0,1,0
1,13.5,2,0,0,1
2,15.3,3,1,0,0


In [58]:
pd.get_dummies(df[['price','color','size']], drop_first=True)

,price,size,color_green,color_red
0,10.1,1,1,0
1,13.5,2,0,1
2,15.3,3,0,0
